# Perform MC simulation on the NPT ensemble using HOOMD-Blue and user defined potential (hard sphere double ramp Jagla model)

### Import necessary libraries and initialise hoomd on CPU.

In [ ]:
import hoomd
import hoomd.hpmc
import hoomd.jit
import hoomd.hpmc.field
import numpy as np
import ase
import ase.io
from matplotlib import pyplot 

In [ ]:
hoomd.context.initialize('--mode=cpu');

### Input parameters: temperature, pressure and initial xyz configuration ###
Note that the temperature is set by scaling the Jagla parameters appropriately - this is down when initialising the potential.
Give the pressure value as in Jagla parameters - it is converted to hoomd units later.

In [ ]:
at=ase.io.read("Jagla_hoomd_LDliq_N585.extxyz")
Temp=0.375 # temperature
Press=0.1  # pressure

In [ ]:
# create HOOMD system
cell = at.get_cell()
pos = at.get_positions()
nd = 1 # if reading in a smaller cell, it can be multiplied to create a larger simulation box.
uc = hoomd.lattice.unitcell(N=len(at),
                            a1=cell[0],
                            a2=cell[1],
                            a3=cell[2],
                            dimensions=3,
                            position=pos,
                            type_name=['A']*len(at));
system = hoomd.init.create_lattice(unitcell=uc, n=[nd, nd, nd]);

In [ ]:
# set HOOMD MC integrator 
# initial stepsize = d
mc = hoomd.hpmc.integrate.sphere(seed=96241, d=0.1)
# hard sphere diameter = diameter
mc.shape_param.set('A', diameter=1.0)
mc.set_params(nselect=1)

### Jagla potential - set parameters

``patch.alpha_iso[0]`` = depth of potential well and ``patch.alpha_iso[1]`` = height of the repulsive ramp. In order to control the temperature of the simulation the calculation corresponds to, both of these parameters are divided by ``Temp``.

``r_cut`` = cutoff distance the potential is truncated 

In [ ]:
# T* = kT/E    
Jagla       = """float rsq = dot(r_ij, r_ij);
                     if (sqrt(rsq) >= 1.72f)    
                         return -alpha_iso[0] + alpha_iso[0]*((sqrt(rsq)-1.72f)/(3.0f-1.72f));
                     else
                         return alpha_iso[1] - ( alpha_iso[1] + alpha_iso[0]) * (sqrt(rsq) - 1.0f)/(1.72f-1.0f);
              """

patch = hoomd.jit.patch.user(mc=mc, r_cut=3.0, array_size=2, code=Jagla)
patch.alpha_iso[0]=1.0/Temp
patch.alpha_iso[1]=3.5/Temp
print(patch.alpha_iso)

In [ ]:
## Alternative potential model: Stepwise version of the Jagla model used e.g. in 
## Luo et al. JOURNAL OF CHEMICAL PHYSICS 142, 224501 (2015)
#
## T* = kT/E    
#Jagla_step  = """float rsq = dot(r_ij, r_ij);
#                     if (sqrt(rsq) >= 1.8f)
#                         return -alpha_iso[0] + ((ceil((100.0*sqrt(rsq)-180.0)/16)))*0.125;
#                     else if (sqrt(rsq) > 1.72f && sqrt(rsq) < 1.80f)
#                         return -alpha_iso[0]; 
#                     else
#                         return alpha_iso[1]-((ceil((100.0*sqrt(rsq)-100.0)/2))-1)*0.125;
#              """
#
#patch = hoomd.jit.patch.user(mc=mc, r_cut=3.0, array_size=2, code=Jagla_step)
#patch.alpha_iso[0]=1.0/Temp
#patch.alpha_iso[1]=3.5/Temp
#print(patch.alpha_iso)

In [ ]:
# quantities to be logged during MC run
quantities=["hpmc_patch_energy","volume","hpmc_overlap_count","pressure",'lx','ly','lz']
logfilename="LDliq_N585.out"
log = hoomd.analyze.log(filename=logfilename,quantities=quantities,period=100)

In [ ]:
# set constant pressure and allow change of the simulation box
betap = Press/Temp # see HOOMD manual: betap=p/(k_t*T)
boxmc = hoomd.hpmc.update.boxmc(mc, betaP=betap, seed=74)

In [ ]:
# set the stepsize for atom moves
mc.set_params(d=0.13)
# volume moves, stepsize controlled by delta, weight is the frequency of this type of box change
boxmc.volume(delta=10.0,weight=1.0)
# box length moves, stepsize controlled by a delta in each dimension, 
# weight is the frequency of this type of box change. If zero, the shape of the box is kept constant.
boxmc.length(delta=(0.0,0.0,0.0), weight=0.0)

### Run simulation ###

In [ ]:
hoomd.run(500)

In [ ]:
# current state of the simulation
U = log.query(quantity="hpmc_patch_energy") # potential energy (this is scaled by Temp!)
OC = log.query(quantity="hpmc_overlap_count") # overlap count of hard spheres (should be zero) 
V = log.query(quantity="volume") # volume of simulation box 
print(U, OC, V)

In [ ]:
data = np.genfromtxt(fname=logfilename, skip_header=True)
pyplot.figure(figsize=(8,4), dpi=100)
pyplot.plot(data[:,2]/len(at))
pyplot.grid(color='k', linestyle='-', linewidth=0.1)
pyplot.xlabel('MC step/100')
pyplot.ylabel('Volume/atom')

In [ ]:
def save_config(hoomd_system,atom_types=["H"]):
    
    lattice=np.array([hoomd_system.box.get_lattice_vector(i=i) for i in range(3)])
    x2 = int(lattice[[0],[0]]) / 2
    y2 = int(lattice[[1],[1]]) / 2
    z2 = int(lattice[[2],[2]]) / 2

    ase_atoms=ase.Atoms(pbc=[(True,True,True)],cell=lattice)
    
    for i in range(system.particles.types.pdata.getN()):
        i_type = hoomd_system.particles.types.pdata.getType(i)
        i_pos = hoomd_system.particles.pdata.getPosition(i)
        i_pos.x = i_pos.x + x2 ; i_pos.y = i_pos.y + y2 ; i_pos.z = i_pos.z + z2
        ase_atoms.append(ase.Atom(atom_types[i_type],position=[i_pos.x,i_pos.y,i_pos.z]))
    return ase_atoms

In [ ]:
current_config = save_config(system)
ase.io.write("Jagla_LDliq_N585.final.extxyz", current_config)